In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 200     # number of variables
num_ineq = 200    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 9256.08
Epoch 1, Validation Loss: 5.83
Epoch 2, Validation Loss: -5.34
Epoch 3, Validation Loss: -11.95
Epoch 4, Validation Loss: -9.65
Epoch 5, Validation Loss: -16.03
Epoch 6, Validation Loss: -16.77
Epoch 7, Validation Loss: -14.30
Epoch 8, Validation Loss: -20.32
Epoch 9, Validation Loss: -20.84
Epoch 10, Validation Loss: -17.06
Epoch 11, Validation Loss: -20.72
Epoch 12, Validation Loss: -20.92
Epoch 13, Validation Loss: -21.56
Epoch 14, Validation Loss: -20.54
Epoch 15, Validation Loss: -23.40
Epoch 16, Validation Loss: -18.04
Epoch 17, Validation Loss: -22.97
Epoch 18, Validation Loss: -22.69
Epoch 19, Validation Loss: -22.58
Epoch 20, Validation Loss: -22.23
Epoch 21, Validation Loss: -3.72
Epoch 22, Validation Loss: -19.28
Epoch 23, Validation Loss: -18.49
Epoch 24, Validation Loss: -19.90
Epoch 25, Validation Loss: -21.93
Epoch 26, Validation Loss: -22.11
Epoch 27, Validation Loss: -21.75
Epoch 28, Validation Loss: -24.17
Epoch 29, Validation Loss: -

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_200-200_g.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  9.00it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -25.349781               0.0      0.003269
std      0.326771               0.0      0.000825
min    -25.937414               0.0      0.001897
25%    -25.591185               0.0      0.002955
50%    -25.411941               0.0      0.003027
75%    -25.179264               0.0      0.003528
max    -23.943559               0.0      0.006057
Number of infeasible solution: 0


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5004.20
Epoch 1, Validation Loss: -3.16
Epoch 2, Validation Loss: -12.77
Epoch 3, Validation Loss: -16.67
Epoch 4, Validation Loss: -17.72
Epoch 5, Validation Loss: -18.83
Epoch 6, Validation Loss: -20.12
Epoch 7, Validation Loss: -20.37
Epoch 8, Validation Loss: -19.77
Epoch 9, Validation Loss: -19.66
Epoch 10, Validation Loss: -22.37
Epoch 11, Validation Loss: -22.23
Epoch 12, Validation Loss: -23.27
Epoch 13, Validation Loss: -20.85
Epoch 14, Validation Loss: -22.08
Epoch 15, Validation Loss: -22.51
Epoch 16, Validation Loss: -24.24
Epoch 17, Validation Loss: -23.72
Epoch 18, Validation Loss: -17.79
Epoch 19, Validation Loss: -17.98
Epoch 20, Validation Loss: -22.19
Epoch 21, Validation Loss: -22.32
Epoch 22, Validation Loss: -24.34
Epoch 23, Validation Loss: -24.69
Epoch 24, Validation Loss: -23.47
Epoch 25, Validation Loss: -21.81
Epoch 26, Validation Loss: -23.09
Epoch 27, Validation Loss: -23.43
Epoch 28, Validation Loss: -19.90
Epoch 29, Validation Los

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_200-200_g.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.93it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -25.110320               0.0      0.003259
std      1.050248               0.0      0.000967
min    -26.056704               0.0      0.001998
25%    -25.697376               0.0      0.002554
50%    -25.296642               0.0      0.003011
75%    -24.899618               0.0      0.003616
max    -19.103955               0.0      0.006130
Number of infeasible solution: 0
